# Module 8 — Classical & Modern Computer Vision Tasks (Expanded)

This notebook covers:

- edge detection (Canny) and Hough lines
- ORB feature detection and descriptor matching (example using synthetic images)
- extracting CNN embeddings and visualizing with t-SNE

Run cells sequentially in Colab. Uses OpenCV and scikit-learn for t-SNE.

## 1 — Setup (install packages and imports)

In [ ]:
!pip -q install -U opencv-python-headless scikit-learn matplotlib --quiet

import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
print('OpenCV version:', cv2.__version__)


## 2 — Edge detection (Canny) and Hough Line Transform

In [ ]:
# Create synthetic image with lines and shapes
img = np.zeros((200,300), dtype=np.uint8)
cv2.rectangle(img, (20,50), (280,150), 255, -1)
cv2.line(img, (0,0), (299,199), 0, 5)

edges = cv2.Canny(img, 50, 150)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)

# visualize
plt.figure(figsize=(10,4))
plt.subplot(1,3,1); plt.imshow(img, cmap='gray'); plt.title('Original'); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(edges, cmap='gray'); plt.title('Canny edges'); plt.axis('off')
# draw lines
line_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
if lines is not None:
    for x1,y1,x2,y2 in lines[:,0]:
        cv2.line(line_img, (x1,y1),(x2,y2),(0,0,255),2)
plt.subplot(1,3,3); plt.imshow(line_img[:,:,::-1]); plt.title('Hough Lines'); plt.axis('off')
plt.show()


## 3 — ORB feature detection and descriptor matching (synthetic pair)

In [ ]:
# Create two slightly different synthetic images
img1 = np.zeros((200,200), dtype=np.uint8)
cv2.circle(img1, (100,100), 50, 255, -1)
img2 = img1.copy()
# add a small translation and rotation to img2
M = cv2.getRotationMatrix2D((100,100), 15, 1.0)
img2 = cv2.warpAffine(img2, M, (200,200))

# ORB detect and compute
orb = cv2.ORB_create(nfeatures=200)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# BFMatcher with Hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)[:30]

# draw matches
match_img = cv2.drawMatches(cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR), kp1, cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR), kp2, matches, None, flags=2)
plt.figure(figsize=(10,6)); plt.imshow(match_img[:,:,::-1]); plt.title('ORB Matches'); plt.axis('off'); plt.show()


## 4 — Extract CNN embeddings and visualize with t-SNE (CIFAR-10 subset)

In [ ]:
# Use a small CNN encoder on CIFAR-10 to get embeddings, then apply t-SNE
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models

(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# use small subset
x = x_train[:2000].astype('float32')/255.0
y = y_train[:2000].flatten()

# build a small encoder model
encoder = models.Sequential([
    layers.Input(shape=(32,32,3)),
    layers.Conv2D(32,3,activation='relu', padding='same'), layers.MaxPooling2D(),
    layers.Conv2D(64,3,activation='relu', padding='same'), layers.MaxPooling2D(),
    layers.Flatten(), layers.Dense(128, activation='relu')
])
embeddings = encoder.predict(x, batch_size=64)
print('Embeddings shape:', embeddings.shape)

# t-SNE (compute on CPU; may take ~30s for 2000 samples)
print('Running t-SNE (this may take a moment)...')
tsne = TSNE(n_components=2, init='pca', random_state=42, perplexity=30, n_iter=1000)
X2 = tsne.fit_transform(embeddings)

# plot with colors
plt.figure(figsize=(8,6))
for cls in np.unique(y):
    idx = np.where(y==cls)
    plt.scatter(X2[idx,0], X2[idx,1], s=8, label=str(int(cls)))
plt.legend(title='class'); plt.title('t-SNE of CNN embeddings (CIFAR-10 subset)')
plt.show()


## 5 — Simple image similarity using embeddings (nearest neighbors)

In [ ]:
# Use embeddings computed above to find nearest neighbors for a sample image
from sklearn.metrics.pairwise import euclidean_distances

dist = euclidean_distances(embeddings)
query_idx = 5
nn_idx = np.argsort(dist[query_idx])[:6]

plt.figure(figsize=(10,4))
for i, idx in enumerate(nn_idx):
    plt.subplot(1,6,i+1)
    plt.imshow(x[idx])
    plt.title('idx='+str(idx))
    plt.axis('off')
plt.suptitle('Nearest neighbors in embedding space for query index '+str(query_idx))
plt.show()


## 6 — Exercises & Instructor Notes

- Replace ORB with SIFT (if OpenCV contrib available) and compare matching performance.
- Try UMAP as an alternative to t-SNE for faster embedding visualization (pip install umap-learn).
- For large datasets, use FAISS for efficient nearest neighbor search instead of brute-force.
- Discuss failures of classical feature matching vs learned descriptors and when to use each approach.